# Importing modules:

In [9]:
import obspy
import os
import glob
import numpy as np
import pandas as pd
import subprocess


import time
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

# Importing pandas dataframe and images (results):

In [10]:
FOLDER_OUTPUT = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/'

In [11]:
images_folder = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/EVENT_MSEED_TELESEISMIC_DETECTION_SELECTED_FINAL/'
images_to_plot = glob.glob(images_folder+'*/*/*.png')

In [12]:
images_to_plot

['/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/EVENT_MSEED_TELESEISMIC_DETECTION_SELECTED_FINAL/ak135/P/event_mseed_detection_ak135_P_2017_06_14_07_29_07_G170E_201706140729A_mag_6.95mw.png',
 '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/EVENT_MSEED_TELESEISMIC_DETECTION_SELECTED_FINAL/ak135/P/event_mseed_detection_ak135_P_2017_04_15_08_19_48_G170B_201704150819A_mag_6.27mw.png',
 '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/EVENT_MSEED_TELESEISMIC_DETECTION_SELECTED_FINAL/ak135/P/event_mseed_detection_ak135_P_2016_11_04_16_20_49_G1607_201611041620A_mag_6.33mw.png',
 '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/EVENT_MSEED_TELESEISMIC_DETECTION_SELECTED_FINAL/ak135/PKP/event_mseed_detection_ak135_PKP_2018_10_13_11_10_26_G180R_201810131110A_mag_6.69mw.png',
 '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/EVENT_MSEED_TELESEISMIC_DETECTION_SELECTED_FINAL/ak135/PKP/event_mseed_detection_ak135_PKP_2019_07_14_09_11_04_G190X_20190714

In [13]:
dataframe_file = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/STATISTICS/table_events_selected.csv'
df_events = pd.read_csv(dataframe_file)

In [14]:
df_events.columns

Index(['Unnamed: 0', 'filename_mseed', 'name_glider', 'Phase', 'ev_ID',
       'ev_time', 'ev_lat', 'ev_long', 'ev_depth', 'distance', 'magnitude',
       'magnitude_type', 'region', 'moment_tensor', 'phases_time',
       'time_glider', 'lat_glider', 'lon_glider', 'dep_glider',
       'time_dive_info', 'lat_dive_info', 'lon_dive_info', 'depth_dive_info',
       'srn_per_band', 'sta_lta_max_per_band', 'noise_per_band',
       'automatic_event_detection', 'automatic_time_residual', 'year',
       'ev_depth_round'],
      dtype='object')

# Creating PDF files from mosaic results:

In [17]:
def create_pdf_figure_mosaic(fig_location,P_label,name_glider,ev_ID,fig_number,event_Date,magnitude, magnitude_type,region):
                                                                         
    os.makedirs(FOLDER_OUTPUT+'FIGURAS/SUPP_MATERIAL/'+P_label+'/',exist_ok=True)
    fig_name_output = FOLDER_OUTPUT+'FIGURAS/SUPP_MATERIAL/'+P_label+'/event_mseed_detection_'+P_label+'_'+event_Date.strftime('%Y_%m_%d_%H_%M_%S')+'_'+name_glider+'_'+ev_ID+'_mag_'+str(magnitude)+magnitude_type+'_S'+str(fig_number)+'.pdf'
    print(fig_name_output)                   
    doc = SimpleDocTemplate(fig_name_output,pagesize=letter,rightMargin=72,leftMargin=72,topMargin=72,bottomMargin=18)
    Story=[]
   
    im = Image(fig_location, 6*inch, 5*inch)
    Story.append(im)
    
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
        
    Story.append(Spacer(1, 24))
    ptext = 'Figure S%s: <b>Waveforms, spectrograms, time series windows centered on the theoretical arrival times, and STA/LTA ratio curves.</b> \
    Mosaic showing the analysis of the magnitude %s %s earthquake (id: %s) that occurred on %s at %s.  \
    Upper panel: Global map showing station-event locations, ray path of the seismic phase, and focal mechanism (colors indicate event depth).  \
    Trajectories of the ocean glider and zoomed-in views at the moment of the record in latitude-longitude and depth-longitude plots.  \
    Lower panel: wide-window waveform filtered (black and grey lines are waveforms filtered below and above 3 Hz, respectively, and the amplitude is presented in digital units) and its spectrogram.  \
    Vertical dashed lines (red/white) and the letter indicate the theoretical arrival time estimated using the ak135f model.  \
    Spectral analysis using five frequency bands centered on the theoretical wave arrival (between -30 and 30 seconds), waveforms, and STA/LTA ratio curves for each band and the raw data.  \
    The red dot indicates the estimated phase arrival based on SNR > 1.5 and STA/LTA ratio > 3 in the last two frequency bands.  \
    The dashed oranges horizontal lines represent the preset lower (1.5) and upper (3) limits for the STA/LTA ratio trigger, and the orange dot is the maximum value.' % (fig_number,magnitude,magnitude_type,ev_ID,event_Date.strftime("%B %d, %Y, at %H:%M:%S"),region.title())
    
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    
    doc.build(Story)

In [18]:
for id,ev in enumerate(df_events.iterrows()):
    
    fig_location = glob.glob(images_folder+'*/*/*'+ev[1]['ev_ID']+'*')[0]

    create_pdf_figure_mosaic(fig_location,ev[1]['Phase'],ev[1]['name_glider'],ev[1]['ev_ID'],id+9,obspy.UTCDateTime(ev[1]['ev_time']),ev[1]['magnitude'],ev[1]['magnitude_type'],ev[1]['region'])

/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/SUPP_MATERIAL/P/event_mseed_detection_P_2017_06_14_07_29_07_G170E_201706140729A_mag_6.95mw_S9.pdf
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/SUPP_MATERIAL/P/event_mseed_detection_P_2017_04_15_08_19_48_G170B_201704150819A_mag_6.27mw_S10.pdf
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/SUPP_MATERIAL/P/event_mseed_detection_P_2016_11_04_16_20_49_G1607_201611041620A_mag_6.33mw_S11.pdf
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/SUPP_MATERIAL/PKP/event_mseed_detection_PKP_2018_10_13_11_10_26_G180R_201810131110A_mag_6.69mw_S12.pdf
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/SUPP_MATERIAL/PKP/event_mseed_detection_PKP_2019_07_14_09_11_04_G190X_201907140910A_mag_7.19mw_S13.pdf
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/SUPP_MATERIAL/PKP/event_mseed_detection_PKP_2019_07_11_17_08_38_G190X_201907111708A_mag_6.0mw_S14.pdf
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGUR

# Creating PDF files from comparison results:

In [20]:
images_comparison_folder = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/EVENTS/FIGURES/REGIONAL/'
images_to_plot = glob.glob(images_comparison_folder+'*.png')

In [21]:
def create_pdf_figure_comparison(fig_location,P_label,name_glider,ev_ID,fig_number,event_Date,magnitude, magnitude_type,region):
                                                                         
    os.makedirs(FOLDER_OUTPUT+'FIGURAS/SUPP_MATERIAL_COMPARISON/'+P_label+'/',exist_ok=True)
    fig_name_output = FOLDER_OUTPUT+'FIGURAS/SUPP_MATERIAL_COMPARISON/'+P_label+'/event_mseed_comparison_'+P_label+'_'+event_Date.strftime('%Y_%m_%d_%H_%M_%S')+'_'+name_glider+'_'+ev_ID+'_mag_'+str(magnitude)+magnitude_type+'_S'+str(fig_number)+'.pdf'
    doc = SimpleDocTemplate(fig_name_output,pagesize=letter,rightMargin=72,leftMargin=72,topMargin=72,bottomMargin=18)
    Story=[]
   
    im = Image(fig_location, 4*inch, 8*inch)
    Story.append(im)
    
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
        
    #Story.append(Spacer(1, 12))
    if P_label == 'P':
        ptext = 'Figure S%s: <b>Comparison of waveforms between ocean gliders and RSBR inland stations.</b> \
                Comparison of waveforms from the glider (black line) and the Brazilian Seismographic Network stations (grey lines) \
                for the magnitude %s %s earthquake (id: %s) on %s UTC  at %s. The waveforms were filtered using a bandpass filter between 0.7 and 2 Hz. \
                Theoretical travel times of P waves according to the ak135f model (land station) and modified ak135f model (glider) are indicated by \
                red circles.' % (fig_number,magnitude,magnitude_type,ev_ID,event_Date.strftime("%B %d, %Y, at %H:%M:%S"),region.title())

    else:
        ptext = 'Figure S%s: <b>Comparison of waveforms between ocean gliders and RSBR inland stations.</b> \
                Comparison of waveforms from the glider (black line) and the Brazilian Seismographic Network stations (grey lines) \                for the magnitude %s %s earthquake (id: %s) on %s UTC  at %s. The waveforms were filtered using a bandpass filter between 0.7 and 2 Hz. \
                Theoretical travel times of P waves according to the ak135f model (land station) and modified ak135f model (glider) are indicated by \
                red circles (PKIKP) and orange circles (PKP).' % (fig_number,magnitude,magnitude_type,ev_ID,event_Date.strftime("%B %d, %Y, at %H:%M:%S"),region.title())

    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    
    doc.build(Story)

In [22]:
for id,ev in enumerate(df_events.iterrows()):
    
    fig_location = glob.glob('/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/EVENTS/FIGURES/REGIONAL/'+'*'+ev[1]['ev_ID']+'*')[0]
    print(fig_location)
    create_pdf_figure_comparison(fig_location,ev[1]['Phase'],ev[1]['name_glider'],ev[1]['ev_ID'],id+25,obspy.UTCDateTime(ev[1]['ev_time']),ev[1]['magnitude'],ev[1]['magnitude_type'],ev[1]['region'])

/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/EVENTS/FIGURES/REGIONAL/event_mseed_2017-06-14T07:29:04_201706140729A.png
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/SUPP_MATERIAL_COMPARISON/P/event_mseed_comparison_P_2017_06_14_07_29_07_G170E_201706140729A_mag_6.95mw_S25.pdf
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/EVENTS/FIGURES/REGIONAL/event_mseed_2017-04-15T08:19:43_201704150819A.png
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/SUPP_MATERIAL_COMPARISON/P/event_mseed_comparison_P_2017_04_15_08_19_48_G170B_201704150819A_mag_6.27mw_S26.pdf
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/EVENTS/FIGURES/REGIONAL/event_mseed_2016-12-08T17:39:00_201611041620A.png
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/FIGURAS/SUPP_MATERIAL_COMPARISON/P/event_mseed_comparison_P_2016_11_04_16_20_49_G1607_201611041620A_mag_6.33mw_S27.pdf
/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/EVENTS/FIGURES/REGIONAL/event_mseed_2018-10-13T11:10:26_201810131110A.png
/hom